Вариаант 21

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
file_data = pd.read_excel("./data8/Volgmed_2013.xlsx", header=1)
data = file_data[['Пол', 'Курс', 'Сила правой руки, кг', 'Рост, см', 'Сила левой руки, кг', 'Бег 100 м, с']]
data = data.dropna()
data = data.loc[((data['Пол'] == 'жен') | (data['Пол'] == 'жен.'))& (data['Курс'] == 1)]
data

In [22]:
def clean_strings(data, column_name):
    data =  data[pd.to_numeric(data[column_name], errors='coerce').notnull()]
    data = data.dropna(axis=0, subset=column_name)
    return data

def clean_data(data, column_name):
    data_arr = data[column_name].to_numpy()
    q_025 = np.quantile(data_arr, 0.25)
    q_075 = np.quantile(data_arr, 0.75)
    iqd = q_075-q_025
    upper_threshold = q_075+1.5*iqd
    lower_threshold = q_025-1.5*iqd
    data =  data.loc[ (data[column_name] <= upper_threshold) & (data[column_name] >= lower_threshold)]
    return data

data = clean_strings(data, 'Сила правой руки, кг')
data = clean_strings(data, 'Сила левой руки, кг')
data = clean_strings(data, 'Рост, см')
data = clean_strings(data, 'Бег 100 м, с')

data = clean_data(data, 'Сила правой руки, кг')
data = clean_data(data, 'Сила левой руки, кг')
data = clean_data(data, 'Рост, см')
data = clean_data(data, 'Бег 100 м, с')

In [23]:
right = data['Сила правой руки, кг'].to_numpy(dtype=float)
X1 = sm.add_constant(data['Рост, см'].to_numpy(dtype=float))
X2 = np.column_stack((X1, data['Сила левой руки, кг'].to_numpy(dtype=float)))
X3 = np.column_stack((X2, data['Бег 100 м, с'].to_numpy(dtype=float)))
model = sm.OLS(right, X3)
res = model.fit()
print(res.summary())
print('Коэффициент детерминации', res.rsquared)
print('Коэффициенты с их оценкой пи-значений:')
names = ['Константа', 'Рост, см', 'Сила левой руки, кг', 'Бег 100 м, с']
params_to_delete = []
for i in range(len(res.params)):
    print(f"{names[i]}: {res.params[i]} (p-значение: {res.pvalues[i]})")
    if(res.pvalues[i] > 0.05):
        params_to_delete.append(i)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.647
Model:                            OLS   Adj. R-squared:                  0.646
Method:                 Least Squares   F-statistic:                     402.3
Date:                Thu, 31 Oct 2024   Prob (F-statistic):          2.91e-148
Time:                        18:53:51   Log-Likelihood:                -1906.5
No. Observations:                 661   AIC:                             3821.
Df Residuals:                     657   BIC:                             3839.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -7.3877      5.124     -1.442      0.1

In [25]:
print(X3)
print(params_to_delete)
for p in params_to_delete:
    print(names)
    X3 = np.delete(X3, p, axis=1)
    names = np.delete(names, p, axis=None)
    params_to_delete = list(np.asarray(params_to_delete) - 1)

model = sm.OLS(right, X3)
res = model.fit()
print(res.summary())
print('Коэффициент детерминации', res.rsquared)
print('Коэффициенты с их оценкой пи-значений:')
for i in range(len(res.params)):
    print(f"{names[i]}: {res.params[i]} (p-значение: {res.pvalues[i]})")

[[176.   34.   17.7]
 [166.   18.   18.7]
 [167.   28.   15. ]
 ...
 [162.5  23.   16.4]
 [158.   15.   17.1]
 [168.2  21.   18.8]]
[-1, 2]
['Рост, см' 'Сила левой руки, кг' 'Бег 100 м, с']
['Рост, см' 'Сила левой руки, кг']


IndexError: index 2 is out of bounds for axis 1 with size 2

In [19]:
residuals = res.resid
sample_variance = np.var(residuals, ddof=1)

mse_list = []
for i in range(200): 
    X_train, X_test, y_train, y_test = train_test_split(X3, right, test_size=0.2)
    X_train_df = pd.DataFrame(X_train, columns=['Рост, см', 'Сила левой руки, кг', 'Бег 100 м, с']) 
    X_test_df = pd.DataFrame(X_test, columns=['Рост, см', 'Сила левой руки, кг', 'Бег 100 м, с']) 
    model = sm.OLS(y_train, X_train_df)
    model = model.fit()
    y_pred = model.predict(X_test_df)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)
mean_mse = np.mean(mse_list)
residuals = model.resid
sample_variance = np.var(residuals, ddof=1)
print(f"Среднее значение квадрата отклонения: {mean_mse}")
print(f"Выборочная дисперсия остатков: {sample_variance}")
print(f"Отношение MSE к выборочной дисперсии: {mean_mse / sample_variance}")

ValueError: Shape of passed values is (528, 2), indices imply (528, 3)